In [52]:
lines = sc.textFile("data/coupon150720.csv")

In [53]:
lines.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [54]:
lines.take(5)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [72]:
lines.cache()

data/coupon150720.csv MapPartitionsRDD[32] at textFile at NativeMethodAccessorImpl.java:-2

In [64]:
airlines = [l.split(",")[4] for l in lines]

TypeError: 'RDD' object is not iterable

In [65]:
airlines = lines.map(lambda l: l.split(",")[4])

In [66]:
airlines.count()

1232662

In [67]:
airlines.cache()

PythonRDD[43] at RDD at PythonRDD.scala:42

In [50]:
len([""]*10)

10

In [71]:
airlines.filter(lambda a: a == "IB").count()

15883

In [73]:
def getBoardOffAmount(l):
    elems  = l.split(",")
    board  = elems[2]
    off    = elems[3]
    route = board + "-" + off
    amount = float(elems[6])
    
    return (route, amount)

In [75]:
getBoardOffAmount(lines.take(1)[0])

(u'MAA-AUH', 56.79)

In [76]:
routes = lines.map(getBoardOffAmount)

In [77]:
routes.take(3)

[(u'MAA-AUH', 56.79), (u'AUH-CDG', 84.34), (u'CJB-MAA', 60.0)]

In [80]:
routes.sortBy(lambda x: x[1],False).take(5)

[(u'SYD-CGK', 6355194.0),
 (u'SYD-CGK', 6355194.0),
 (u'SYD-CGK', 6355194.0),
 (u'CDG-IAD', 637313.52),
 (u'IAD-CDG', 637313.52)]

In [81]:
routes.filter(lambda x: x[1]<5e3).sortBy(lambda x: x[1],False).take(5)

[(u'LHR-MEL', 4985.75),
 (u'LHR-JFK', 4984.45),
 (u'HND-FRA', 4969.62),
 (u'ACC-JFK', 4932.0),
 (u'LHR-RUH', 4926.08)]

In [82]:
#### Exercise to be solved by the students
def getBoardOffAmount2(l):
    elems  = l.split(",")
    board  = elems[2]
    off    = elems[3]    
    route = board + "-" + off
    airline = elems[4]
    booking_class = elems[9]
    amount = float(elems[6])
    
    return (route, airline, booking_class, amount)

In [83]:
lines.map(getBoardOffAmount2).filter(lambda x: x[3]<5e3).sortBy(lambda x: x[3],False).take(5)

[(u'LHR-MEL', u'QF', u'J', 4985.75),
 (u'LHR-JFK', u'BA', u'J', 4984.45),
 (u'HND-FRA', u'LH', u'A', 4969.62),
 (u'ACC-JFK', u'DL', u'C', 4932.0),
 (u'LHR-RUH', u'BA', u'F', 4926.08)]

In [92]:
import math

In [90]:
amount_logs = routes.map(lambda x: math.log(x[1]+1))

In [93]:
limit_value = math.exp(amount_logs.mean() + 3*amount_logs.stdev())

In [94]:
limit_value

8320.431042239863

In [95]:
lines.map(getBoardOffAmount2).filter(lambda x: x[3]<limit_value).sortBy(lambda x: x[3],False).take(5)

[(u'LAX-PVG', u'MU', u'P', 8320.0),
 (u'LAX-LHR', u'BA', u'F', 8263.86),
 (u'DXB-AKL', u'EK', u'J', 8124.71),
 (u'MIA-CCS', u'S3', u'S', 8120.0),
 (u'MIA-CCS', u'S3', u'S', 8120.0)]

In [96]:
airlines2 = lines.map(getBoardOffAmount2)

In [97]:
airlines2.take(5)

[(u'MAA-AUH', u'9W', u'H', 56.79),
 (u'AUH-CDG', u'9W', u'H', 84.34),
 (u'CJB-MAA', u'9W', u'H', 60.0),
 (u'DEL-DXB', u'9W', u'S', 160.63),
 (u'AUH-IXE', u'9W', u'V', 152.46)]

In [100]:
amount_per_airline_class = airlines2.map(lambda x: ((x[0],x[1],x[2]),x[3]))

In [101]:
amount_per_airline_class.take(5)

[((u'MAA-AUH', u'9W', u'H'), 56.79),
 ((u'AUH-CDG', u'9W', u'H'), 84.34),
 ((u'CJB-MAA', u'9W', u'H'), 60.0),
 ((u'DEL-DXB', u'9W', u'S'), 160.63),
 ((u'AUH-IXE', u'9W', u'V'), 152.46)]

In [119]:
def getLimitValue(vs):
    ls = [math.log(v+1) for v in vs]
    m = sum(ls)/len(ls) 
    deviations = [(x - m)**2 for x in ls]
    s = math.sqrt(sum(deviations)/len(deviations))
    return math.exp(m + 3*s)
    

In [120]:
getLimitValue([1,.3,4,5,6,2,6,5])

23.97491363971152

In [121]:
groups = amount_per_airline_class.groupByKey().map(lambda x: (x[0], getLimitValue(x[1])))

In [122]:
groups.take(5)

[((u'TRD-BOO', u'SK', u'V'), 62.68379060561998),
 ((u'DPS-TMC', u'GA', u'N'), 66.72409117823226),
 ((u'BEG-CDG', u'JU', u'K'), 89.11),
 ((u'SAV-CLT', u'US', u'N'), 71.03999999999999),
 ((u'DUS-AMS', u'KL', u'C'), 33.07)]

In [123]:
amount_per_airline_class.take(5)

[((u'MAA-AUH', u'9W', u'H'), 56.79),
 ((u'AUH-CDG', u'9W', u'H'), 84.34),
 ((u'CJB-MAA', u'9W', u'H'), 60.0),
 ((u'DEL-DXB', u'9W', u'S'), 160.63),
 ((u'AUH-IXE', u'9W', u'V'), 152.46)]

In [124]:
joined = amount_per_airline_class.leftOuterJoin(groups)

In [126]:
joined.take(50)

[((u'TRD-BOO', u'SK', u'V'), (42.76, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (48.67, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (42.76, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (42.76, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (42.76, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (48.67, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (42.76, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (48.67, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (42.76, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (42.76, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (42.76, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (42.76, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (42.76, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (29.55, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (42.76, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (42.76, 62.68379060561998)),
 ((u'TRD-BOO', u'SK', u'V'), (29.55, 62.68379060561998)),
 ((u'TRD-BOO',

In [129]:
joined.filter(lambda x: x[1][0] >= x[1][1]).take(5)

[((u'JFK-FCO', u'AZ', u'X'), (304.49, 304.02240331274174)),
 ((u'SIN-HKG', u'CX', u'S'), (681.3, 336.5794532383178)),
 ((u'HAM-IST', u'TK', u'W'), (41.8, 40.356597087525536)),
 ((u'SIN-HKG', u'CX', u'I'), (789.36, 786.6278674028141)),
 ((u'BNE-MEL', u'QF', u'B'), (224.87, 218.56597373122125))]

In [132]:
joined.sortBy(lambda x: x[1][0], False).take(25)

[((u'SYD-CGK', u'GA', u'G'), (6355194.0, 6355195.000000028)),
 ((u'SYD-CGK', u'GA', u'G'), (6355194.0, 6355195.000000028)),
 ((u'SYD-CGK', u'GA', u'G'), (6355194.0, 6355195.000000028)),
 ((u'CDG-IAD', u'AF', u'P'), (637313.52, 637314.5199999998)),
 ((u'IAD-CDG', u'AF', u'P'), (637313.52, 637314.5199999998)),
 ((u'COO-CDG', u'AF', u'J'), (538599.2, 538600.2000000002)),
 ((u'CDG-COO', u'AF', u'J'), (538599.2, 252544360.50641724)),
 ((u'DKR-NBO', u'KQ', u'T'), (173800.0, 14188769.259681763)),
 ((u'NBO-DKR', u'KQ', u'T'), (173800.0, 2332635.813454238)),
 ((u'CDG-DKR', u'AF', u'L'), (161010.22, 161011.21999999986)),
 ((u'CMN-NIM', u'AT', u'R'), (99994.96, 11627495.087685555)),
 ((u'NBO-HAH', u'KQ', u'L'), (72100.0, 28427966.886865743)),
 ((u'NBO-HAH', u'KQ', u'L'), (72100.0, 28427966.886865743)),
 ((u'TSN-ICN', u'KE', u'Z'), (38655.45, 38656.450000000004)),
 ((u'VIE-NRT', u'OS', u'S'), (35704.26, 65666.32933767821)),
 ((u'DEL-CMB', u'UL', u'D'), (21499.82, 3076364.282108203)),
 ((u'CMB-DEL'

In [131]:
joined.filter(lambda x: x[1][0] < x[1][1]).sortBy(lambda x: x[1][0], False).take(25)

[((u'SYD-CGK', u'GA', u'G'), (6355194.0, 6355195.000000028)),
 ((u'SYD-CGK', u'GA', u'G'), (6355194.0, 6355195.000000028)),
 ((u'SYD-CGK', u'GA', u'G'), (6355194.0, 6355195.000000028)),
 ((u'CDG-IAD', u'AF', u'P'), (637313.52, 637314.5199999998)),
 ((u'IAD-CDG', u'AF', u'P'), (637313.52, 637314.5199999998)),
 ((u'COO-CDG', u'AF', u'J'), (538599.2, 538600.2000000002)),
 ((u'CDG-COO', u'AF', u'J'), (538599.2, 252544360.50641724)),
 ((u'DKR-NBO', u'KQ', u'T'), (173800.0, 14188769.259681763)),
 ((u'NBO-DKR', u'KQ', u'T'), (173800.0, 2332635.813454238)),
 ((u'CDG-DKR', u'AF', u'L'), (161010.22, 161011.21999999986)),
 ((u'CMN-NIM', u'AT', u'R'), (99994.96, 11627495.087685555)),
 ((u'NBO-HAH', u'KQ', u'L'), (72100.0, 28427966.886865743)),
 ((u'NBO-HAH', u'KQ', u'L'), (72100.0, 28427966.886865743)),
 ((u'TSN-ICN', u'KE', u'Z'), (38655.45, 38656.450000000004)),
 ((u'VIE-NRT', u'OS', u'S'), (35704.26, 65666.32933767821)),
 ((u'DEL-CMB', u'UL', u'D'), (21499.82, 3076364.282108203)),
 ((u'ICN-BOM'

In [133]:
joined.map(lambda x: x[0][1]).distinct().count()

359

In [134]:
total_revenue = joined.map(lambda x: (x[0][1], x[1][0])).reduceByKey(lambda x,y: x+y)

In [136]:
total_revenue.sortBy(lambda x: x[1],False).take(10)

[(u'GA', 20712226.56999999),
 (u'BA', 12443986.98000006),
 (u'AF', 10221507.909999827),
 (u'QF', 8358587.8999999175),
 (u'LH', 7715608.339999989),
 (u'QR', 6935584.370000024),
 (u'SV', 6286299.339999829),
 (u'UA', 5151815.290000008),
 (u'JJ', 5066300.509999973),
 (u'AA', 4151996.4600000097)]

In [138]:
total_rev_per_pax = joined.map(lambda x: (x[0][1], (x[1][0], 1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))

In [140]:
total_rev_per_pax.map(lambda x: (x[0], x[1][0]/x[1][1])).sortBy(lambda x: x[1], False).take(10)

[(u'S3', 5225.068852459017),
 (u'9V', 1488.6739864864862),
 (u'GA', 991.2527671691787),
 (u'TN', 964.6472580645161),
 (u'7F', 668.035),
 (u'DT', 579.0006976744186),
 (u'B8', 443.295),
 (u'V0', 438.61227730441516),
 (u'NE', 430.1),
 (u'4M', 377.7557471264368)]

In [152]:
## Exercise for the students
## PAX per channel

def getChannel(l):
    elems = l.split(",")
    tcn = elems[0]
    channel = elems[8]
    return (tcn, channel)

In [153]:
channels = sc.textFile('data/transm150720.csv').map(getChannel)

In [154]:
channels.take(5)

[(u'79062005698500', u'T'),
 (u'79062005924069', u'T'),
 (u'79065668570385', u'T'),
 (u'79065668737021', u'T'),
 (u'79062006192650', u'T')]

In [155]:
channels.map(lambda x: (x[1], 1)).reduceByKey(lambda x,y: x+y).take(15)

[(u'A', 360506), (u'', 381), (u'E', 4610), (u'T', 264241), (u'V', 1667)]

In [156]:
## Exercise for the students
## Total revenue per channel
def getTCNAndAmount(l):
    elems  = l.split(",")
    tcn    = elems[0]
    amount = float(elems[6])
    
    return (tcn, amount)

In [157]:
tcn_rev = lines.map(getTCNAndAmount)

In [158]:
tcn_rev.take(5)

[(u'79062005698500', 56.79),
 (u'79062005698500', 84.34),
 (u'79062005924069', 60.0),
 (u'79065668570385', 160.63),
 (u'79065668737021', 152.46)]

In [159]:
channels.take(5)

[(u'79062005698500', u'T'),
 (u'79062005924069', u'T'),
 (u'79065668570385', u'T'),
 (u'79065668737021', u'T'),
 (u'79062006192650', u'T')]

In [160]:
channel_with_rev = channels.leftOuterJoin(tcn_rev)

In [168]:
channel_with_rev.map(lambda x: x[1]).reduceByKey(lambda x,y: x+y).top(2)

[(u'V', 843207.13), (u'T', 83637221.2200021)]